In [35]:
import os
from langchain_community.document_loaders import PyPDFLoader

In [36]:
DOC_PATH = "Annual-Report-for-the-Financial-Year-2023-2024.pdf"
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

In [37]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant

In [38]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [48]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from dotenv import load_dotenv
load_dotenv()

qdrant_client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

In [51]:
# print(os.getenv("QDRANT_API_KEY"))

In [50]:
collection_name = "RAG_PDF"
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

C:\Users\dhruv\AppData\Local\Temp\ipykernel_16836\652732412.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [52]:
vector_store = Qdrant.from_documents(
    documents=pages,
    embedding=embedding_model,
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
    collection_name=collection_name,
)

In [67]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
llm = Ollama(model="llama3.2:3b")
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

query = "Explain  Company Overview in concise"
response = rag_chain(query)

print("Answer:", response["result"])

Answer: Here is a concise explanation of "Company Overview" from the given context:

Tata Industries Limited (TIL) is an investment holding company, part of the promoter group with shareholding in Group companies. It is registered as a Core Investment Company with the Reserve Bank of India and engages in diversified businesses, including Strategy and Management Consulting, Data Analytics, Healthcare Diagnostics, Education Technology, Healthsciences, and Biosciences.
